In [ ]:
!pip install llama-index llama_index.embeddings.huggingface llama-index-readers-file llama-index-llms-groq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Aula 1

In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

from google.colab import userdata
groq = userdata.get('GROQ_API_KEY')


llm = Groq(model="llama-3.3-70b-versatile", api_key=groq)

Settings.llm = llm



In [ ]:
import requests
from bs4 import BeautifulSoup

def buscar_mencoes_sem_api(termo: str) -> str:
    url = f"https://pt.wikipedia.org/wiki/{termo.replace(' ', '_')}"
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, "html.parser")
    resultados = soup.find_all("div", class_="mw-content-container")
    
    lista_resultados = []
    for resultado in resultados[:1]:  # Pegar apenas o primeiro resultado
        titulo = resultado.find("h1").text if resultado.find("h1") else "Sem título"
        # Extrai o texto do primeiro parágrafo (sem tags HTML)
        primeiro_paragrafo = resultado.find("p").get_text() if resultado.find("p") else "Sem texto"
        lista_resultados.append(f"Título: {titulo}\nTexto: {primeiro_paragrafo}")
    
    return "\n\n".join(lista_resultados) if lista_resultados else "Nenhuma menção encontrada."
    
# Exemplo de uso
print(buscar_mencoes_sem_api("Carlos Llamando"))


Título: Calling Carlos: Llamando a Carlos - Books
Link: https://www.amazon.com/Calling-Carlos-Mar%C3%ADa-Victoria-Castillo/dp/1964792010

Título: Calling Carlos: Llamando a Carlos (Paperback)
Link: https://www.booksoup.com/book/9781964792019

Título: Calling Carlos: Llamando a Carlos (Paperback)
Link: https://www.avidbookshop.com/book/9781964792019

Título: Llamando a Carlos by Castillo, MarÍa Victoria, Brand New ...
Link: https://www.ebay.com/itm/196551782966

Título: Calling Carlos: Llamando a Carlos
Link: https://agendabookshop.com/products/calling-carlos-llamando-a-carlos?srsltid=AfmBOoqsbwFW2-tPfrPr-YIwM2eDbMtwjrxSAngrUhVgTrQfJkGi1v2b


In [ ]:
from llama_index.core.tools import FunctionTool

In [ ]:
ferramenta = FunctionTool.from_defaults(fn=buscar_mencoes_sem_api,name="Busca menções",description=(
    """    Busca menções a um termo na web usando o Google Search (web scraping).

    Args:
        termo (str): O termo ou nome que você deseja buscar.

    Returns:
        str: Resultados encontrados (títulos e links) ou uma mensagem indicando que nada foi encontrado."""
    ))

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
# Criando o agente
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta],
    verbose=True,
    allow_parallel_tool_calls=True
)

In [ ]:
from llama_index.core.agent import AgentRunner

In [ ]:
agent = AgentRunner(agent_worker)

# Consultando o agente
response = agent.chat("Quais são as últimas menções sobre Carlos Llamando?")